In [1]:
import findspark
findspark.init('/home/akshay/spark-3.5.1-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_tutorials_1").getOrCreate()

24/06/27 19:20:45 WARN Utils: Your hostname, akshay-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/27 19:20:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/27 19:20:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Understanding `partitionBy` in Apache Spark

In Apache Spark, `partitionBy` is a method used on DataFrames or Datasets to partition data based on one or more columns. Partitioning is a crucial optimization technique in distributed data processing that improves query performance, especially when dealing with large datasets.

### Key Concepts:

1. **Partitioning:** Divides data into groups or partitions based on a specified column or expression. Each partition is processed independently by different nodes (executors) in the Spark cluster.

2. **Usage:**
   - **Data Organization:** `partitionBy` is typically used when writing data to disk or another storage system (like HDFS or Amazon S3). It allows data to be organized into directories or files based on the partitioning column, facilitating efficient data retrieval and processing.
   
   - **Performance:** Partitioning can significantly enhance performance by minimizing data shuffling during joins, aggregations, and other operations. It ensures that related data is co-located within the same partition, reducing the amount of data movement across the cluster.

3. **Syntax:**
   ```scala
   // Example of using partitionBy in Scala
   val df = spark.read.format("csv").load("path/to/data")
   df.write.partitionBy("partition_column").format("parquet").save("output_path")


In [2]:
data = [
    (1, "Alice", "Female", "IT"),
    (2, "Bob", "Male", "Finance"),
    (3, "Charlie", "Male", "HR"),
    (4, "Diana", "Female", "IT"),
    (5, "Eve", "Female", "Finance"),
    (6, "Frank", "Male", "HR"),
    (7, "Grace", "Female", "IT"),
    (8, "Henry", "Male", "Finance"),
    (9, "Ivy", "Female", "HR"),
    (10, "Jack", "Male", "IT"),
    (11, "Katherine", "Female", "Finance"),
    (12, "Leo", "Male", "HR"),
    (13, "Mary", "Female", "IT"),
    (14, "Nick", "Male", "Finance"),
    (15, "Olivia", "Female", "HR"),
    (16, "Peter", "Male", "IT"),
    (17, "Quinn", "Female", "Finance"),
    (18, "Ryan", "Male", "HR")
]

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType

In [5]:
schema = StructType([StructField("id",dataType=IntegerType()), \
                        StructField("name",dataType=StringType()), \
                        StructField("gender",dataType=StringType()), \
                        StructField("dept",dataType=StringType()) \
                    ])

In [6]:
df = spark.createDataFrame(data=data, schema=schema)

In [7]:
df.show()

+---+---------+------+-------+
| id|     name|gender|   dept|
+---+---------+------+-------+
|  1|    Alice|Female|     IT|
|  2|      Bob|  Male|Finance|
|  3|  Charlie|  Male|     HR|
|  4|    Diana|Female|     IT|
|  5|      Eve|Female|Finance|
|  6|    Frank|  Male|     HR|
|  7|    Grace|Female|     IT|
|  8|    Henry|  Male|Finance|
|  9|      Ivy|Female|     HR|
| 10|     Jack|  Male|     IT|
| 11|Katherine|Female|Finance|
| 12|      Leo|  Male|     HR|
| 13|     Mary|Female|     IT|
| 14|     Nick|  Male|Finance|
| 15|   Olivia|Female|     HR|
| 16|    Peter|  Male|     IT|
| 17|    Quinn|Female|Finance|
| 18|     Ryan|  Male|     HR|
+---+---------+------+-------+



In [8]:
df.write.parquet(path="./data/employee_1/", mode="overwrite", partitionBy="dept")

In [9]:
df.write.parquet(path="./data/employee_2/", mode="overwrite", partitionBy=["dept","gender"])

In [13]:
df_1 = spark.read.parquet("./data/employee_2/dept=HR/", schema=schema)
df_1.show()

+---+-------+------+
| id|   name|gender|
+---+-------+------+
|  3|Charlie|  Male|
| 15| Olivia|Female|
|  6|  Frank|  Male|
| 18|   Ryan|  Male|
| 12|    Leo|  Male|
|  9|    Ivy|Female|
+---+-------+------+

